In [ ]:
!pip install dash

In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
df                       =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str,
                                   'Div2Airport': str, 'Div2TailNum': str})

# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[html.H1('Flight Delay Time Statistics', style = {'textAlign':'center', 'color':'#503D36', 'font-size':30}),
                                html.Div(['Input Year :   ', dcc.Input(id = 'inpyear', value = '2010', type = 'number', style = {'height':'35px', 'font-size':30}),], style = {'font-size':30, 'color':'white'}),
                                html.Br(),
                                html.Br(),
                                html.Div([
                                        html.Div(dcc.Graph(id = 'carrierPlot')),
                                        html.Div(dcc.Graph(id = 'weatherPlot'))
                                ], style={'display': 'flex'}),

                                html.Div([
                                        html.Div(dcc.Graph(id = 'nasPlot')),
                                        html.Div(dcc.Graph(id='securityPlot'))
                                ], style={'display': 'flex'}),

                                html.Div(dcc.Graph(id = 'latePlot'), style={'width':'65%'})
                                ])

#create the compute_info function to grouping the data based on the year input
def computeInfo(df, year):
  dfs = df[df['Year'] == int(year)]
  avg_car = dfs.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
  avg_weather = dfs.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
  avg_NAS = dfs.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
  avg_sec = dfs.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
  avg_late = dfs.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
  return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

#callback the decorator
@app.callback([
    Output(component_id = 'carrierPlot', component_property = 'figure'),
    Output(component_id = 'weatherPlot', component_property = 'figure'),
    Output(component_id = 'nasPlot', component_property = 'figure'),
    Output(component_id = 'securityPlot', component_property = 'figure'),
    Output(component_id = 'latePlot', component_property = 'figure'),
], Input(component_id = 'inpyear', component_property = 'value'))

def getGraph(year):
  #compute the some required datas for creating graph to the created function
  avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(df, year)

  #create a line plot for the each data
  carline = px.line(avg_car, x = 'Month', y = 'CarrierDelay', color='Reporting_Airline', title='Average carrier delay time (minutes) by airline')
  wealine = px.line(avg_weather, x = 'Month', y = 'WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
  nasline = px.line(avg_NAS, x = 'Month', y = 'NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
  secline = px.line(avg_sec, x = 'Month', y = 'SecurityDelay', color='Reporting_Airline', title='Average Security delay time (minutes) by airline')
  latline = px.line(avg_late, x = 'Month', y = 'LateAircraftDelay', color='Reporting_Airline', title='Average Late Aircraft delay time (minutes) by airline')

  return [carline, wealine, nasline, secline, latline]

if __name__ == '__main__':
  app.run_server()

<IPython.core.display.Javascript object>